# Plots for GH climate

## Import dependencies and data

In [ ]:
# import dependencies
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
# define relevant paths
home_path = os.path.dirname(os.getcwd())
data_path = home_path + '\\data\\'
plot_path = home_path + '\\plotting\\plots\\climate\\'

In [6]:
# define HYPERPARAMS
plot_unit   = '6 hours'
plot_suffix = '6h'
plot_nsteps = 720 # 2 steps per minute

# set seaborn style/them
sns.set_theme()
sns.set_style('white')

In [9]:
# get merged data
data = pd.read_csv(
    data_path + 'data_merged.csv',
    header=[0, 1],
    index_col=[0, 1, 2, 3]
)

# convert index.date col to datetime
#data.index = pd.to_datetime(data.index.values)
data.index = data.index.set_levels(
    levels=pd.to_datetime(data.index.get_level_values(3).values),
    level=3
)

# drop all index levels except date
data = data.droplevel(
    level=[0, 1, 2], 
    axis=0
)

## Define plot specs

In [10]:
# define plot categories and params
plot_cats = [
    # EXAMPLE: ('title', 'type', 'filter (RegEx)', 'units', 'palette', 'alpha', 'filename'),
    {
        'title':'DC - Temperatures, {}'.format(plot_unit),
        'type':'temperatures',
        'filter':'GM401',
        'units':'$^\circ$C', 
        'palette':'YlOrRd_r',
        'alpha':0.9,
        'fname':'dc_temp_{}.png'.format(plot_suffix)
    },
    {
        'title':'DC - Humidity, {}'.format(plot_unit),
        'type':'humidity',
        'filter':'GM401',
        'units':'$\%$', 
        'palette':'crest',
        'alpha':0.9,
        'fname':'dc_humid_{}.png'.format(plot_suffix)
    },
    {
        'title':'GH - Avg., {}'.format(plot_unit),
        'type':'all',
        'filter':'X',
        'units':'$^\circ$C, $\%$', 
        'palette':'mako',
        'alpha':1.0,
        'fname':'ghavg_all_{}.png'.format(plot_suffix)
    },
    {
        'title':'DC - , {}'.format(plot_unit),
        'type':'all',
        'filter':'DC_GT401_GM401',
        'units':'$^\circ$C, $\%$', 
        'palette':'mako',
        'alpha':1.0,
        'fname':'dc_all_{}.png'.format(plot_suffix)
    },
    {
        'title':'GH & OUT - Temperatures, {}'.format(plot_unit),
        'type':'temperatures',
        'filter':'X|GT301|SMHI',
        'units':'$^\circ$C', 
        'palette':'YlOrRd_r',
        'alpha':1.0,
        'fname':'ghout_temp_{}.png'.format(plot_suffix)
    },
    {
        'title':'GH, DC & OUT - Humidity, {}'.format(plot_unit),
        'type':'humidity',
        'filter':'X|DC_GT401_GM401|SMHI',
        'units':'$\%$', 
        'palette':'crest',
        'alpha':1.0,
        'fname':'ghdcout_hum_{}.png'.format(plot_suffix)
    },
    {
        'title':'OUT - Temperature, {}'.format(plot_unit),
        'type':'temperatures',
        'filter':'GT301|SMHI',
        'units':'$^\circ$C', 
        'palette':'mako',
        'alpha':1.0,
        'fname':'out_temp_{}.png'.format(plot_suffix)
    },
    {
        'title':'GH - Temperature, {}'.format(plot_unit),
        'type':'temperatures',
        'filter':'X',
        'units':'$^\circ$C', 
        'palette':'mako',
        'alpha':1.0,
        'fname':'ghavg_temp_{}.png'.format(plot_suffix)
    }
]

## Create and save figures

In [ ]:
# crop data
data_cropped = data.iloc[-plot_nsteps:]

# iterate over plot params and save figs
for params in plot_cats[-1:]:
    
    if params['type'] == 'all':
        subset = data_cropped.filter(regex=params['filter'], axis=1).droplevel(level=1, axis=1)
    else:
        subset = data_cropped[params['type']].filter(regex=params['filter'])
    ax = sns.relplot(
        data=subset,
        kind='line',
        palette= params['palette'],
        alpha= params['alpha'],
        dashes=False,
        markers=False,
        legend='full'
    )

    ax.set_xticklabels(step=2)
    
    plt.yticks(rotation=45)
    plt.xlabel('Timestamp')
    plt.ylabel(params['units'], loc='center', rotation=0, fontsize=12, labelpad=30)
    plt.title(params['title'])
    
    fpath = plot_path + params['fname']
    plt.savefig(fpath, bbox_inches='tight', dpi=1000)
    plt.show()